# Proyecto Final - Ensemble Learning (Aprendizaje por ensamblado)
## Parte 2 - Implementación y predicción

**Curso:** Statistical Learning

**Catedrático:** Ing. Luis Leal

**Estudiante:** Dany Rafael Díaz Lux (21000864)

**Objetivo:** Hacer clasificación binaria para determinar si una persona sobrevive o no del hundimiento del Titanic.

In [1]:
# Import required libraries
#from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
#import datetime as dt
import joblib
#import matplotlib.pylab as plt
#import numpy as np 
#import os.path
import pandas as pd
import sklearn.metrics as mts
#import tensorflow as tf
#print('Tensor flow version: ' + tf.__version__)

In [4]:
# Informacion de modelos
directorioModelos = 'modelos/'
extensionModelos = '.modelo'
nombreModeloArbolDecision = 'ArbolDecision_criterio=gini_profundidadMaxima=4'

In [5]:
# Cargar modelos
modeloArbolDecision = joblib.load(directorioModelos + nombreModeloArbolDecision + extensionModelos)

In [8]:
pruebas = pd.read_csv("datos_validacion_temp.csv")
caracteristicas = pruebas.iloc[:,:-1]
etiquetas = pruebas.iloc[:,-1]

In [10]:
# Revisar métricas de datos de prueba
predicciones = modeloArbolDecision.predict(caracteristicas)
print(mts.accuracy_score(etiquetas, predicciones))
print(mts.f1_score(etiquetas, predicciones))
print(mts.precision_score(etiquetas, predicciones))
print(mts.recall_score(etiquetas, predicciones))

0.8321678321678322
0.8095238095238094
0.8225806451612904
0.796875


## Función de predicción

## Predicción árbol de decisión

## Predicción Support Vector Machine (SVM)

## Predicción Naive Bayes

## Predicción regresión logística

## Validación